In [1]:
from wordcloud import WordCloud,STOPWORDS
import matplotlib as plt
import scipy
import numpy as np
import pandas as pd
import json
import re

In [15]:
punc = re.compile(r"[\n,!\?\’'\+:\"\.\$&@#/\(\)\[\]\|\{\}]")
nonwords = re.compile('^[^a-zA-Z0-9]+$')
links = re.compile(r"https")

In [16]:
def clean_text(df, col_to_clean: str): 
    df[col_to_clean] = df[col_to_clean].str.lower().replace('\n','',regex=True).str.replace('’',"'",regex=True).str.replace(punc,'',regex=True).str.strip()
    df = df[~df[col_to_clean].str.contains(links)]
    df = df[~df[col_to_clean].str.contains(nonwords)]
    df = df[df[col_to_clean]!='']
    return df

def tokinize(df, col_to_tokenize: str):
    df['tokenized'] = df[col_to_tokenize].str.split(' ')
    return df

def freqs(df, col_tok='tokenized'):
    df = df.explode(col_tok)
    freqs = df[col_tok].value_counts().to_dict()
    STOPWORDS.update({punc.sub('',x) for x in STOPWORDS})
    new_freqs = {word:freq for (word,freq) in freqs.items() if word not in STOPWORDS}
    return new_freqs

In [20]:
def get_post_df(json_filename):
    df = pd.read_json(json_filename, lines=True)
    # changing the type col
    df = df.rename({"_type": "type"}, axis="columns")
    df = df.replace(to_replace="snscrape.modules.reddit.Submission", value="post")
    df = df.replace(to_replace="snscrape.modules.reddit.Comment", value="comment")
    # change to posts only
    df_posts = df[df["type"] == "post"]
    df_posts = df_posts.dropna(subset=["selftext"])
    df_posts = df_posts.drop(["body"], axis=1)
    df_posts = df_posts.drop(["type", "parentId", "subreddit", "link"], axis=1)
    df_posts = df_posts.reset_index()

    return df, df_posts

def apply_text_len(df, col_text='selftext', minimum_length=500, visualise_lengths=False):
    df['text_len'] = df[col_text].apply(len)
    if(visualise_lengths):
        plt2 = df['text_len'].plot(kind='kde',
            title="Distribution of Comment Length",
            xlabel='Comment Length',
            xlim=0)
    df = df[df["text_len"] >= minimum_length]
    return df

In [ ]:
_, doomer_posts = get_post_df("Data/r_doomer_scraped.json")
# _, lonely_posts = get_post_df("Data/r_lonely_scraped.json")
# _, depr_posts = get_post_df("Data/r_depression_scraped.json")

In [21]:
doomer_posts = apply_text_len(doomer_posts)

In [17]:
doomer_posts = clean_text(doomer_posts, 'selftext')
doomer_posts = tokinize(doomer_posts, 'selftext')
doomer_freqs = freqs(doomer_posts)

{'': 5953,
 'life': 2121,
 'people': 1644,
 'nothing': 1622,
 'feel': 1616,
 'know': 1479,
 'even': 1391,
 'want': 1309,
 'will': 1233,
 'real': 1232,
 'time': 1195,
 'one': 1176,
 'now': 1054,
 'think': 975,
 'never': 917,
 'really': 893,
 'go': 870,
 'going': 761,
 'friends': 760,
 'years': 746,
 'good': 742,
 'day': 715,
 'still': 713,
 'way': 710,
 'make': 708,
 'things': 700,
 'something': 686,
 'much': 678,
 'everything': 662,
 'see': 661,
 'world': 645,
 'always': 627,
 'back': 598,
 'shit': 584,
 'fucking': 581,
 'love': 561,
 'better': 541,
 'got': 540,
 'every': 539,
 'work': 531,
 'anything': 524,
 'school': 481,
 'live': 477,
 'alone': 459,
 'end': 459,
 'someone': 457,
 'find': 436,
 'try': 433,
 'thing': 422,
 'say': 418,
 'around': 416,
 'everyone': 407,
 'right': 405,
 'job': 395,
 'take': 384,
 'talk': 382,
 'bad': 378,
 'hate': 376,
 'year': 375,
 'away': 374,
 'need': 374,
 'maybe': 370,
 'last': 365,
 'happy': 364,
 'fuck': 364,
 'many': 362,
 'getting': 362,
 'fami

In [23]:
cloud = WordCloud(stopwords=STOPWORDS,height=1000,width=2000).generate_from_text(''.join(doomer_posts['selftext']))

In [ ]:
newfreqs.pop("")

In [ ]:
punc = re.compile(r"[\n,!\?\’'\+:\"\.\$&@#/\(\)\[\]\|\{\}]")
plt.figure(figsize=(8,5),dpi=200)
plt.axis('off')
plt.imshow(cloud,interpolation='bilinear')

In [ ]:
plt.figure(figsize=(7,5),dpi=400)
plt.axis("off")
plt.imshow(cloud,interpolation='bilinear')

In [ ]:
df['body_length'] = df['body'].apply(len)

In [ ]:
plt2 = doomer_df['text_len'].plot(kind='kde',
    title="Distribution of Comment Length",
    xlabel='Comment Length',
    xlim=0)

In [ ]:
df[df['body_length']>500]['body']

In [ ]:
df['tokenized'] = df['body'].astype(str).str.lower().str.split(" ")
df = df.explode('tokenized')
freqs = df['tokenized'].value_counts().to_dict()
newfreqs = {k:v for k,v in freqs.items() if k not in STOPWORDS}


df = clean_text(df)

In [10]:
doomer_comments.assign(text_len=doomer_comments['body'].astype(str).map(len))
doomer_posts.assign(text_len=doomer_posts['selftext'].astype(str).map(len))

doomer_df = pd.concat([doomer_posts, doomer_comments])

In [ ]:
doomer_df['te']

In [11]:
ave_txt_len = int(doomer_df[['text_len']].mean(axis=0))
shortest_txt_len = int(min(doomer_df['text_len']))
longest_txt_len = int(max(doomer_df['text_len']))

KeyError: "None of [Index(['text_len'], dtype='object')] are in the [columns]"

In [ ]:
plt2 = doomer_df['text_len'].plot(kind='kde',
    title="Distribution of Comment Length",
    xlabel='Comment Length',
    xlim=0)

In [14]:
lonely_comments = doomer_comments[doomer_comments['body'].str.contains('lonely')]


In [15]:
lonely_posts = doomer_posts[doomer_posts['selftext'].astype(str).str.contains('lonely')]

In [17]:
lonely_posts.reset_index(drop=True,inplace=True)

In [22]:
import re

In [41]:
import nltk

In [ ]:
nltk.

In [24]:
lonely = re.compile(".{40}lonely.{40}")

In [29]:
from collections import Counter

In [36]:
df = pd.Series(Counter(text.split()))

In [40]:
df.nlargest(25)

and        73
I          49
to         49
her        44
a          35
she        28
was        27
the        26
in         21
that       14
being      14
with       14
for        14
time       13
it         12
me         12
be         11
of         11
had        11
She        11
at         11
is         10
talking    10
this        9
all         9
dtype: int64

In [34]:
Counter(text.split())

Counter({'To': 1,
         'explain': 1,
         'my': 4,
         'story': 2,
         'I': 49,
         'will': 1,
         'start': 1,
         'from': 2,
         'the': 26,
         'beginning.': 1,
         'met': 4,
         'a': 35,
         'girl': 4,
         'online': 4,
         'and': 73,
         'we': 7,
         'really': 7,
         'hit': 4,
         'it': 12,
         'off.': 1,
         'know': 6,
         'meeting': 2,
         'girls': 1,
         'is': 10,
         'considered': 1,
         'cringy': 1,
         'pathetic': 1,
         'but': 3,
         'have': 6,
         'mental': 1,
         'impairment.': 1,
         'Aspergers': 1,
         'disorder': 2,
         'which': 2,
         'basically': 3,
         'condition': 1,
         'that': 14,
         'makes': 1,
         'you': 2,
         'hard': 3,
         'time': 13,
         'socialising': 1,
         'being': 14,
         'comfortable': 1,
         'around': 2,
         'others.': 1,
         'So

In [28]:
text.split()

['To',
 'explain',
 'my',
 'story',
 'I',
 'will',
 'start',
 'from',
 'the',
 'beginning.',
 'I',
 'met',
 'a',
 'girl',
 'online',
 'and',
 'we',
 'really',
 'hit',
 'it',
 'off.',
 'I',
 'know',
 'meeting',
 'girls',
 'online',
 'is',
 'considered',
 'cringy',
 'and',
 'pathetic',
 'but',
 'I',
 'have',
 'a',
 'mental',
 'impairment.',
 'I',
 'have',
 'Aspergers',
 'disorder',
 'which',
 'is',
 'basically',
 'a',
 'condition',
 'that',
 'makes',
 'you',
 'have',
 'a',
 'really',
 'hard',
 'time',
 'socialising',
 'and',
 'being',
 'comfortable',
 'around',
 'others.',
 'So',
 'to',
 'finally',
 'be',
 'able',
 'to',
 'hit',
 'it',
 'of',
 'with',
 'a',
 'girl',
 'that',
 'looked',
 'very',
 'attractive',
 'to',
 'me',
 'and',
 'easy',
 'to',
 'talk',
 'to',
 'felt',
 'really',
 'nice.',
 'We',
 'hit',
 'things',
 'off',
 'so',
 'well',
 'that',
 'we',
 'decide',
 'to',
 'be',
 'in',
 'a',
 'online',
 'relationship',
 'together',
 'and',
 'decided',
 'we',
 'where',
 'in',
 'love',
 

In [26]:
lonely.findall(text)

[' many hours being there when they where lonely and had nobody and locked in some shi*t']

In [25]:
text = lonely_posts.head()['selftext'][0]